In [1]:
using Flux , Distributions , DifferentialEquations
using Base.Iterators: repeated
using Flux.Data.MNIST
using NeuralNetDiffEq

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/bb/.julia/packages/CUDAdrv/b1mvw/src/CUDAdrv.jl:67
┌ Info: Precompiling NeuralNetDiffEq [8faf48c0-8b73-11e9-0e63-2155955bfa4d]
└ @ Base loading.jl:1273
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Package NeuralNetDiffEq does not have Distributions in its dependencies:
│ - If you have NeuralNetDiffEq checked out for development and have
│   added Distributions as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with NeuralNetDiffEq
└ Loading Distributions into NeuralNetDiffEq from project dependency, future warnings for NeuralNetDiffEq are suppressed.


In [2]:
function phi(x)
    return sum(x.^2, dims=1)
end        

phi (generic function with 1 method)

In [3]:
d = 10
T = 1.0
num_samples = 1000


1000

In [4]:
xspan = (0.0 , 1.0)
tspan = (0.0,5.0)
prob = KolmogorovPDEProblem(phi , xspan , tspan)

opt = Flux.ADAM(0.1, (0.9, 0.95))



ADAM(0.1, (0.9, 0.95), IdDict{Any,Any}())

In [5]:
chain = Flux.Chain(Dense(10,128,tanh),Dense(128,256 ,tanh ) , Dense(256,256 ,tanh) , Dense(256,128 ,tanh) , Dense(128 , 1) )

Chain(Dense(10, 128, tanh), Dense(128, 256, tanh), Dense(256, 256, tanh), Dense(256, 128, tanh), Dense(128, 1))

In [27]:
loss(x , y) =Flux.mse(chain(x), y)
function evalcbw()
    @show loss(x_sde , y)
end

evalcbw (generic function with 1 method)

In [30]:
dataset = repeated((x_sde, y), 1000)
max_err = 10000
opt = ADAM()
Flux.train!(loss, Flux.params(chain),dataset, opt, cb = evalcbw)

LoadError: syntax: missing comma or ) in argument list

In [36]:
dataset = repeated((x_sde, y), 1000)


Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float64,2},Array{Float64,2}}}}(Base.Iterators.Repeated{Tuple{Array{Float64,2},Array{Float64,2}}}(([1.7452384960544745 0.8224755080200252 … -1.201648978488635 -0.3853798865653628; 0.13897368619641595 2.8211116814557684 … 1.627722206456363 -0.10547871487213137; … ; 1.919863005744676 -0.22548537376255784 … 0.6433964240941288 1.7012963425815713; 0.48282144736015165 2.487263691802881 … 1.6503486907597265 1.6703269417483886], [27.893370131157525 29.181951580763315 … 17.00239466407266 24.30968601681051])), 1000)

In [17]:
chain(x_sde)

1×1000 Array{Float32,2}:
 23.5582  23.5599  23.5533  23.5614  …  23.5473  23.5615  23.518  23.5575

In [18]:
y

1×1000 Array{Float64,2}:
 14.1758  18.3955  19.8112  36.9394  …  21.752  30.5465  44.4972  14.2022

In [40]:
l = Uniform(0 , 1)

Uniform{Float64}(a=0.0, b=1.0)

In [6]:
sol = solve(prob, NeuralNetDiffEq.NNKolmogorov(chain,opt), verbose = true, dt = 0,
            abstol=1e-10, maxiters = 10)

UndefKeywordError: UndefKeywordError: keyword argument dt not assigned

In [4]:
prob


KolmogorovPDEProblem
timespan: (0.0, 5.0)xspan: (0.0, 1.0)phi